# Pipeline prototype – classify, retrieve, respond

## Environment setup

In [3]:
!pip -q install -U \
  qdrant-client \
  sentence-transformers \
  langgraph \
  langchain-core \
  openai \
  transformers \
  accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.4/477.4 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.5 MB/s eta 0:00:00


## Mount Google Drive

In [4]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Config and OpenAI client

In [12]:
import os
from openai import OpenAI

# ===== Qdrant local persistence path on Drive (folder) =====
QDRANT_PATH = "/content/drive/MyDrive/VibeQ-EIE/DB/qdrant_db"
QDRANT_COLLECTION = "action_cards"

# ===== Classifier path or module =====
# Option A (HF folder): a folder that contains config.json / model.safetensors etc.
CLASSIFIER_HF_DIR = "/content/drive/MyDrive/VibeQ-EIE/models/student_distilled_9_v2"  # <-- change or set None

# ===== Embedding model used for Qdrant vectors =====
EMBED_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

# ===== LLM =====
OPENROUTER_API_KEY = "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" # @param {type:"string"}

WRITER_MODEL = "gpt-4o-mini"   # or your OpenRouter qwen model string
SAFETY_MODEL = "gpt-4o-mini"   # can be smaller/cheaper, but keep it reliable

llm = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

# ===== Retrieval =====
TOP_K = 5
MIN_SCORE = 0.15  # similarity threshold; tune later

# ===== Your emotion labels =====
EMOTION_LABELS = ["joy","sadness","anger","disgust","fear","caring","anticipation","surprise","neutral"]


## Embedding model

In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np

embedder = SentenceTransformer(EMBED_MODEL_NAME)

def embed_text(text: str) -> list[float]:
    vec = embedder.encode([text], normalize_embeddings=True)[0]
    return vec.astype(np.float32).tolist()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Qdrant client

In [7]:
from qdrant_client import QdrantClient

client = QdrantClient(path=QDRANT_PATH)

# Quick sanity checks
print("Collections:", [c.name for c in client.get_collections().collections])


Collections: ['action_cards']


## Retrieval helper

In [25]:
from qdrant_client.http.models import Filter, FieldCondition, MatchValue, Range

def retrieve_action_cards(query: str, primary_emotion: str, top_k: int = TOP_K):
    qvec = embed_text(query)

    flt = Filter(
        must=[
            FieldCondition(key="risk_level", match=MatchValue(value="low")),
            # Only retrieve cards strongly targeting the primary emotion
            FieldCondition(
                key=f"emotion_targets.{primary_emotion}",
                range=Range(gte=0.5)   # tune 0.4–0.7
            )
        ]
    )

    res = client.query_points(
        collection_name=QDRANT_COLLECTION,
        query=qvec,
        limit=top_k,
        query_filter=flt,
        with_payload=True,
        score_threshold=MIN_SCORE,
    )

    # Depending on client version, res may be a tuple or an object with .points
    points = res.points if hasattr(res, "points") else res[0]

    results = []
    for p in points:
        payload = dict(p.payload or {})
        payload["_score"] = float(p.score)
        results.append(payload)

    return results


## Emotion classifier setup

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

tokenizer = None
clf_model = None

if CLASSIFIER_HF_DIR:
    tokenizer = AutoTokenizer.from_pretrained(CLASSIFIER_HF_DIR)
    clf_model = AutoModelForSequenceClassification.from_pretrained(CLASSIFIER_HF_DIR).to(device)
    clf_model.eval()
    print("Loaded classifier from:", CLASSIFIER_HF_DIR)
else:
    print("CLASSIFIER_HF_DIR is None. Use CLASSIFIER_PY_FILE option or plug your own classifier.")


Device: cpu
Loaded classifier from: /content/drive/MyDrive/VibeQ-EIE/models/student_distilled_9_v2


## Emotion prediction helper

In [10]:
import numpy as np

def predict_emotions(journal_text: str):
    """
    Returns:
      {
        "primary": "fear",
        "secondary": ["surprise", "anticipation"],
        "probs": {"fear":0.84, ...}   # sums to 1 if softmax; if multi-label, not necessarily
      }
    """
    if clf_model is None or tokenizer is None:
        raise RuntimeError("Classifier not loaded. Set CLASSIFIER_HF_DIR or plug custom classifier.")

    inputs = tokenizer(
        journal_text,
        truncation=True,
        max_length=256,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        out = clf_model(**inputs)
        logits = out.logits[0].detach().cpu().numpy()

    # Choose ONE of these depending on your head:
    # A) Single-label classifier (softmax)
    probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()

    # B) Multi-label classifier (sigmoid) — if you trained that:
    # probs = torch.sigmoid(torch.tensor(logits)).numpy()

    probs_dict = {EMOTION_LABELS[i]: float(probs[i]) for i in range(len(EMOTION_LABELS))}
    primary = max(probs_dict, key=probs_dict.get)
    secondary = [k for k,_ in sorted(probs_dict.items(), key=lambda x: x[1], reverse=True) if k != primary][:2]

    return {"primary": primary, "secondary": secondary, "probs": probs_dict}


## Build retrieval query

In [11]:
def build_retrieval_query(journal_text: str, clf_out: dict) -> str:
    primary = clf_out["primary"]
    secondary = clf_out["secondary"]
    # Keep journal short for embedding stability
    short = journal_text.strip().replace("\n"," ")
    short = short[:700]

    # Add emotion hints + “coping skill steps” keywords
    return (
        f"{short}\n\n"
        f"emotions: {primary}, {', '.join(secondary)}\n"
        f"task: retrieve coping action steps, grounding, breathing, communication, planning\n"
    )


## Writer LLM

In [13]:
def writer_llm(journal_text: str, clf_out: dict, cards: list[dict]) -> str:
    primary = clf_out["primary"]
    secondary = clf_out["secondary"]

    # Provide only limited card content so the model stays grounded
    cards_compact = []
    for c in cards[:6]:
        cards_compact.append({
            "id": c.get("id"),
            "title": c.get("title"),
            "summary": c.get("summary"),
            "when_to_use": c.get("when_to_use"),
            "when_to_avoid": c.get("when_to_avoid"),
            "steps": c.get("steps"),
            "micro_script": c.get("micro_script"),
            "risk_level": c.get("risk_level"),
            "score": c.get("_score"),
        })

    system = (
        "You are a supportive, practical coping assistant. "
        "Use ONLY the provided action cards to suggest coping steps. "
        "Do NOT invent new techniques. "
        "Be brief, calm, non-judgmental. "
        "Output must include: (1) 1 short validation line, (2) pick 1-2 action cards, "
        "(3) show the steps exactly, (4) mention 1 relevant 'when_to_avoid' if exists, (5) ask at most 1 question."
    )

    user = {
        "journal": journal_text,
        "predicted_emotions": {"primary": primary, "secondary": secondary},
        "action_cards": cards_compact
    }

    resp = llm.chat.completions.create(
        model=WRITER_MODEL,
        messages=[
            {"role":"system", "content": system},
            {"role":"user", "content": str(user)},
        ],
        temperature=0.4,
    )
    return resp.choices[0].message.content


## Safety checker

In [14]:
def safety_llm(draft: str) -> str:
    system = (
        "You are a safety checker for mental-wellbeing responses. "
        "Ensure the response is supportive and does not encourage harm. "
        "If the draft includes dangerous instructions, remove them and replace with safer alternatives. "
        "Keep the tone calm. Keep it short."
    )
    resp = llm.chat.completions.create(
        model=SAFETY_MODEL,
        messages=[
            {"role":"system","content":system},
            {"role":"user","content":draft},
        ],
        temperature=0.2,
    )
    return resp.choices[0].message.content


## Graph wiring

In [26]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Any, List, Dict

class State(TypedDict, total=False):
    journal: str
    clf: Dict[str, Any]
    query: str
    retrieved: List[Dict[str, Any]]
    draft: str
    final: str

def node_classifier(state: State) -> State:
    clf_out = predict_emotions(state["journal"])
    return {"clf": clf_out}

def node_querybuilder(state: State) -> State:
    q = build_retrieval_query(state["journal"], state["clf"])
    return {"query": q}

def node_retriever(state: State) -> State:
    primary = state["clf"]["primary"]
    cards = retrieve_action_cards(state["query"], primary_emotion=primary, top_k=TOP_K)
    return {"retrieved": cards}


def node_writer(state: State) -> State:
    draft = writer_llm(state["journal"], state["clf"], state["retrieved"])
    return {"draft": draft}

def node_safety(state: State) -> State:
    safe = safety_llm(state["draft"])
    return {"final": safe}

graph = StateGraph(State)
graph.add_node("classifier", node_classifier)
graph.add_node("querybuilder", node_querybuilder)
graph.add_node("retriever", node_retriever)
graph.add_node("writer", node_writer)
graph.add_node("safety", node_safety)

graph.set_entry_point("classifier")
graph.add_edge("classifier", "querybuilder")
graph.add_edge("querybuilder", "retriever")
graph.add_edge("retriever", "writer")
graph.add_edge("writer", "safety")
graph.add_edge("safety", END)

app = graph.compile()
print("Graph compiled.")


Graph compiled.


## Quick test run

In [27]:
test_journal = """
Tomorrow I have an interview and I can’t stop thinking about everything that can go wrong.
My heart feels fast and my mind is racing.
"""

out = app.invoke({"journal": test_journal})
print("Predicted:", out["clf"])
print("\n--- Retrieved cards (top 5) ---")
for c in out["retrieved"][:5]:
    print(c.get("id"), c.get("_score"), "-", c.get("title"))

print("\n--- FINAL ---\n")
print(out["final"])


Predicted: {'primary': 'fear', 'secondary': ['sadness', 'anger'], 'probs': {'joy': 0.004370572045445442, 'sadness': 0.1061774492263794, 'anger': 0.004817396402359009, 'disgust': 0.0011796837206929922, 'fear': 0.8720284104347229, 'caring': 0.000981892109848559, 'anticipation': 0.0033551736269146204, 'surprise': 0.003968589473515749, 'neutral': 0.0031208752188831568}}

--- Retrieved cards (top 5) ---
act_fear_54321_grounding_002 0.436157347335886 - 5-4-3-2-1 Grounding
act_fear_worry_postpone_003 0.4301030657368694 - Worry Postponement (10-minute window)
act_fear_boxbreathing_001 0.41540368809669315 - Box Breathing (4×4)

--- FINAL ---

It's completely normal to feel anxious before an interview. Let's take some steps to help you manage those feelings.

**Action Cards:**
1. **5-4-3-2-1 Grounding**
   - Name 5 things you can see.
   - Name 4 things you can feel (touch).
   - Name 3 things you can hear.
   - Name 2 things you can smell.
   - Name 1 thing you can taste or are grateful for.

2